In [1]:
from datasets import load_dataset
ds = load_dataset("rajpurkar/squad")

README.md:   0%|          | 0.00/7.62k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/14.5M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/1.82M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/87599 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10570 [00:00<?, ? examples/s]

In [2]:
train_dataset=[]
train_dataset_length=ds["train"].num_rows
for index in range(10000):
    train_dataset.append({"context":ds["train"][index]["context"],"question":ds["train"][index]["question"]})

In [3]:
import torch
from transformers import AdamW
from torch.utils.data import DataLoader, Dataset

class QADataset(Dataset):
    def __init__(self, data, tokenizer, max_length=512):
        self.data = data
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        context = self.data[idx]['context']
        question = self.data[idx]['question']
        encoding_context = self.tokenizer(
            context,
            truncation=True,
            max_length=self.max_length,
            padding="max_length",
            return_tensors="pt"
        )
        
        encoding_question = self.tokenizer(
            question,
            truncation=True,
            max_length=self.max_length,
            padding="max_length",
            return_tensors="pt"
        )
        
        input_ids = encoding_context['input_ids'].flatten()
        labels =encoding_question['input_ids'].flatten()
        attention_mask = encoding_context['attention_mask'].flatten()  

        return {
            'input_ids': input_ids,
            'labels': labels,
            'attention_mask': attention_mask 
        }





The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

In [4]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch
device=torch.device("cuda" if torch.cuda.is_available() else "cpu")
tokenizer = AutoTokenizer.from_pretrained("google-t5/t5-base")
model = AutoModelForSeq2SeqLM.from_pretrained("google-t5/t5-base").to(device)

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [ ]:
import torch
torch.cuda.empty_cache()

In [ ]:
#!nvidia-smi

In [ ]:
#!pip install bitnet

In [ ]:
#from bitnet import replace_linears_in_hf 
#replace_linears_in_hf(model)

In [5]:
qg_dataset = QADataset(train_dataset, tokenizer)
train_loader = DataLoader(qg_dataset, batch_size=4, shuffle=True)

In [6]:
import torch
from transformers import AdamW
model.train()
optimizer = AdamW(model.parameters(), lr=5e-5)
epochs = 1
for epoch in range(epochs):
    for batch in train_loader:
        input_ids = batch['input_ids'].to(device)
        labels = batch['labels'].to(device)
        attention_mask = batch['attention_mask'].to(device) 
        
        optimizer.zero_grad()
        outputs = model(input_ids, labels=labels, attention_mask=attention_mask)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        
    print(f"Epoch {epoch + 1}/{epochs}, Loss: {loss.item()}")


model.save_pretrained("./fine_tuned_t5_qg")
tokenizer.save_pretrained("./fine_tuned_t5_qg")


/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch 1/1, Loss: 0.059901099652051926


('./fine_tuned_t5_qg/tokenizer_config.json',
 './fine_tuned_t5_qg/special_tokens_map.json',
 './fine_tuned_t5_qg/spiece.model',
 './fine_tuned_t5_qg/added_tokens.json',
 './fine_tuned_t5_qg/tokenizer.json')

In [7]:
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
tokenizer = AutoTokenizer.from_pretrained("/kaggle/working/fine_tuned_t5_qg")
model = AutoModelForSeq2SeqLM.from_pretrained("/kaggle/working/fine_tuned_t5_qg")
model.eval()
def generate_question(context):
    inputs = tokenizer.encode(context, return_tensors='pt')
    attention_mask = (inputs != tokenizer.pad_token_id).long() 
    outputs = model.generate(inputs, attention_mask=attention_mask, max_length=100, num_return_sequences=1)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)



context = """
Notre Dame is known for its competitive admissions, with the incoming class enrolling in fall 2015 admitting 3,577 from a pool of 18,156 (19.7%). The academic profile of the enrolled class continues to rate among the top 10 to 15 in the nation for national research universities. The university practices a non-restrictive early action policy that allows admitted students to consider admission to Notre Dame as well as any other colleges to which they were accepted. 1,400 of the 3,577 (39.1%) were admitted under the early action plan. Admitted students came from 1,311 high schools and the average student traveled more than 750 miles to Notre Dame, making it arguably the most representative university in the United States. While all entering students begin in the College of the First Year of Studies, 25% have indicated they plan to study in the liberal arts or social sciences, 24% in engineering, 24% in business, 24% in science, and 3% in architecture.
"""
question = generate_question(context)
print(f"question: {question}")


question: What percentage of the incoming class admitted to Notre Dame in fall 2015?


In [8]:
import zipfile
import os
folder_to_compress = '/kaggle/working/fine_tuned_t5_qg'  
zip_file_name = 'QGCheckpoint.zip' 
with zipfile.ZipFile(zip_file_name, 'w', zipfile.ZIP_DEFLATED) as zip_file:
    for root, dirs, files in os.walk(folder_to_compress):
        for file in files:
            file_path = os.path.join(root, file)
            zip_file.write(file_path, os.path.relpath(file_path, folder_to_compress))

print(f'{zip_file_name} created successfully!')


QGCheckpoint.zip created successfully!
